# BERTopic: 
- Preprocess (normalize text, filter for "yes" label)

- Embedding (convert text to number representation)

- Top Modeling (find different cluster setups)

In [21]:
#imports / installs

import pandas as pd
import re
import glob
import os


## Step 1: Preprocessing

In [49]:
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)
    # Remove mentions and hashtags
    text = re.sub(r"@\w+|#\w+", "", text)
    # Remove non-letter characters (keep punctuation if needed)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [50]:
from sentence_transformers import SentenceTransformer

def generate_embeddings(texts, model_name="all-MiniLM-L6-v2"):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(texts, show_progress_bar=True)
    return embeddings


In [51]:
INPUT_PATH = "../../data/climate_classified/" #Currently uses the 14 jsons * 100,000 posts 

json_pattern = os.path.join(INPUT_PATH,'*.json')
combined_paths = glob.glob(json_pattern)

dfs = []


for path in combined_paths:
    try:
        df = pd.read_json(path)
        dfs.append(df)
    except ValueError as e:
        print(f"Failed to read {path}: {e}")


In [52]:
if dfs:
    temp = pd.concat(dfs, ignore_index=True)
    print(f"Loaded {len(temp)} posts from {len(dfs)} files.")
else:
    print("No data loaded.")

Loaded 1311270 posts from 14 files.


### NOTE: Currently filters for 60+ characters and score >= 0.99 !

In [53]:
# Step 1: Filter for climate-related posts only
climate_df = temp[temp["label"] == "yes"].copy()

# Step 2: Light preprocessing
climate_df["clean_text"] = climate_df["text"].astype(str).apply(preprocess_text)

# Step 3: Filter on character length and score
climate_df = climate_df[
    
    (climate_df["clean_text"].str.len() >= 60) &
    (climate_df["score"] >= 0.99)
].copy()

print(f"Remaining posts after full filtering: {len(climate_df)}")

Remaining posts after full filtering: 11467


In [54]:
# Save filtered climate_df

output_path = "../../data/filtered/above60chars_above99score.json"
climate_df.to_json(output_path)

print(f"Filtered dataset saved to: {output_path}")

Filtered dataset saved to: ../../data/filtered/above60chars_above99score.json


In [55]:
climate_df.columns

Index(['repo', 'seq', 'text', 'timestamp', 'cid', 'uri', 'label', 'score',
       'clean_text'],
      dtype='object')

In [56]:
climate_df.head()

,repo,seq,text,timestamp,cid,uri,label,score,clean_text
25,did:plc:uli2rqyfqasvuawksu2z5jkc,7778280581,Trump's executive order trying to block state ...,2025-04-09 21:10:45.855,bafyreihbjn7mnkbiytl4wc2jjhukux7xfncg772auwhhu...,at://did:plc:uli2rqyfqasvuawksu2z5jkc/app.bsky...,yes,0.997684,trumps executive order trying to block state c...
204,did:plc:4zh2idecxr5zudhn3oniodhw,7778286641,Spain and Canada signed agreements on renewabl...,2025-04-09 21:10:53.664,bafyreibvwoj6qzbffnpz4rkgxena26ejvpfqoznkbed7n...,at://did:plc:4zh2idecxr5zudhn3oniodhw/app.bsky...,yes,0.993054,spain and canada signed agreements on renewabl...
411,did:plc:m6ntt433rso3lp7dxaja3mue,7778293323,When did you bitch about what Republicans were...,2025-04-09 21:11:03.083,bafyreic4vipeqxz6mlm36qa7uw3qqjbxwo3qtvucl2fek...,at://did:plc:m6ntt433rso3lp7dxaja3mue/app.bsky...,yes,0.996543,when did you bitch about what republicans were...
441,did:plc:cm4nhw2xk43bczonk7mbfvrb,7778294643,"Hydrogen, as you know, is useful for decarboni...",2025-04-09 21:11:05.110,bafyreidsl6kwy2dx6rtvqkhjyafcffv6cwdud7aefrubf...,at://did:plc:cm4nhw2xk43bczonk7mbfvrb/app.bsky...,yes,0.997304,hydrogen as you know is useful for decarbonisi...
448,did:plc:ci5fsjcdjgoct5k3yllky4ud,7778294887,Either we end the Fossil Fuel Era or the Fossi...,2025-04-09 21:11:05.321,bafyreifegyoen4hku664cni3qqh3v6xbnoptueyb76dqm...,at://did:plc:ci5fsjcdjgoct5k3yllky4ud/app.bsky...,yes,0.995790,either we end the fossil fuel era or the fossi...


## Step 2: Embedding Generation